In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 849kB 5.3MB/s 
     |████████████████████████████████| 829kB 10.0MB/s 
     |████████████████████████████████| 184kB 20.5MB/s 
     |████████████████████████████████| 276kB 23.2MB/s 
     |████████████████████████████████| 112kB 26.1MB/s 
     |████████████████████████████████| 1.3MB 27.9MB/s 
     |████████████████████████████████| 296kB 48.1MB/s 
     |████████████████████████████████| 143kB 47.5MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=cc7b16ec2d2b822f900b73d185c73ef0328aca114b78c85e3d09da46845576b7
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=eb8c319d5346ae9ec339f7cf48d6074ecf8499af4ac761841f7a4136368753d4
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future

In [ ]:
import torch
import torchvision
import pytorch_lightning as pl

In [ ]:

class MNISTDataModule(pl.LightningDataModule):

    def __init__(self, path = '../data', batch_size = 64):
        super().__init__()
        self.path = path
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.mnist_train = torchvision.datasets.MNIST(
            self.path, train=True, download=True, transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
          )
        self.mnist_val = torchvision.datasets.MNIST(
            self.path, train=False, download=True, transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
          )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_val, batch_size=self.batch_size)



In [ ]:
from pytorch_lightning.metrics.functional.classification import accuracy
import torch.nn.functional as F

def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )
class Modelo(pl.LightningModule):

    def __init__(self, n_channels=1, n_outputs=10):
        super().__init__()
        self.conv1 = block(n_channels, 64)
        self.conv2 = block(64, 128)
        self.fc = torch.nn.Linear(128*7*7, n_outputs)
        self.train_acc = pl.metrics.Accuracy()
        self.val_acc = pl.metrics.Accuracy()
        self.train_precision = pl.metrics.Precision(num_classes=n_outputs)
        self.val_precision = pl.metrics.Precision(num_classes=n_outputs)
        self.train_recall = pl.metrics.Recall(num_classes=n_outputs)
        self.val_recall = pl.metrics.Recall(num_classes=n_outputs)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('loss', loss)
        self.log('acc', self.train_acc(y_hat, y), prog_bar=True)
        self.log('precision', self.train_precision(y_hat, y), prog_bar=True)
        self.log('recall', self.train_recall(y_hat, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.val_acc(y_hat, y), prog_bar=True)
        self.log('val_precision', self.val_precision(y_hat, y), prog_bar=True)
        self.log('val_recall', self.val_recall(y_hat, y), prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

callbacks durante el entrenamiento puedo modificar por eejemplo si la accuracy no mejora en los epochs


In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping #earlystoopinf
from pytorch_lightning.callbacks import ModelCheckpoint

early_stop_callback = EarlyStopping(
   monitor='val_acc', # el valor en el modelo
   patience=3, #cuantas epoch tiene que pasar seguida sin que mejore
   verbose=False,
   mode='max'
)

#### chepoint es para guardar el mejor modelo entrenado
checkpoint = ModelCheckpoint(
    dirpath='./',              #donde queremos que guarde el modelo
    filename='modelo-{val_acc:.5f}', #nombre y ademas las variables que queramos
    save_top_k=1, #el numero de mejores moedlos que guarde
    monitor='val_acc', # la metrica a memorizar
    mode='max'  #el valor mas grande modo max
)                                                                             



modelo = Modelo()
dm = MNISTDataModule()

trainer = pl.Trainer(
    gpus=1,
    callbacks=[early_stop_callback,checkpoint ]
)

trainer.fit(modelo, dm)


In [ ]:
#### cargando el mejor modelo para hacer predicciones y lo que sea

modelo = Modelo.load_from_checkpoint(checkpoint_path="modelo-val_acc=0.99060.ckpt")